In [1]:
# !pip install -q langchain langchain_community langchain-google-genai chromadb google-api-python-client


In [2]:
!pip install -U langchain langchain-core langchain-community langchain-google-genai chromadb google-api-python-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 20.5 MB/

In [3]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["GOOGLE_API_KEY_GENAI"] = user_secrets.get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = user_secrets.get_secret("GOOGLE_API_KEY_CONSOLE")
os.environ["GOOGLE_CSE_ID"] = user_secrets.get_secret("GOOGLE_CSE_ID")



In [4]:
data = [
  {
    "id": "001",
    "event": "US Imposes 104% Tariff on Chinese Imports",
    "date": "2025-04-09",
    "region": "USA",
    "summary": "US Imposes 104% Tariff on Chinese Imports. President Trump's administration has implemented a 104% tariff on Chinese imports, escalating trade tensions between the two nations.",
    "source": "https://www.businessinsider.com/trump-tariffs-are-here-2025-4",
    "category": "trade policy"
  },
  {
    "id": "002",
    "event": "China Retaliates with 84% Tariff on US Goods",
    "date": "2025-04-09",
    "region": "China",
    "summary": "China Retaliates with 84% Tariff on US Goods. In response to US tariffs, China has announced an 84% tariff on US goods, effective April 10, 2025.",
    "source": "https://www.aljazeera.com/news/liveblog/2025/4/9/trump-tariffs-stocks-dive-as-world-braces-for-duties-to-begin-at-midnight",
    "category": "trade policy"
  },
  {
    "id": "003",
    "event": "India Unlikely to Retaliate Against US Tariffs",
    "date": "2025-04-06",
    "region": "India",
    "summary": "India Unlikely to Retaliate Against US Tariffs. Indian officials indicate that India does not plan to retaliate against the 26% tariff imposed by the US, citing progress in trade deal talks.",
    "source": "https://www.reuters.com/world/india-unlikely-retaliate-against-trumps-tariffs-deal-talks-progress-sources-say-2025-04-06/",
    "category": "trade policy"
  },
  {
    "id": "004",
    "event": "EU Seeks Zero Tariff from India on Car Imports",
    "date": "2025-04-07",
    "region": "European Union",
    "summary": "EU Seeks Zero Tariff from India on Car Imports. In the wake of US tariff hikes, the European Union is negotiating with India for zero tariffs on car imports to strengthen trade ties.",
    "source": "https://www.reuters.com/business/autos-transportation/after-trump-eu-seeks-zero-tariff-india-car-imports-sources-say-2025-04-07/",
    "category": "trade policy"
  },
  {
    "id": "005",
    "event": "Oil Prices Plummet Amid US-China Tariff Escalation",
    "date": "2025-04-09",
    "region": "Global",
    "summary": "Oil Prices Plummet Amid US-China Tariff Escalation. Oil prices dropped nearly 4% following the US’s imposition of 104% tariffs on Chinese goods and China's immediate retaliation.",
    "source": "https://www.reuters.com/business/energy/oil-slides-nearly-4-us-kicks-off-104-tariffs-china-2025-04-09/",
    "category": "economic impact"
  },
  {
    "id": "006",
    "event": "India's Exports to US Could Decline by $5.76 Billion Due to Tariffs",
    "date": "2025-04-07",
    "region": "India",
    "summary": "India's Exports to US Could Decline by $5.76 Billion Due to Tariffs. Analysts estimate India may lose $5.76 billion in export revenue to the US in 2025 due to rising tariff barriers.",
    "source": "https://timesofindia.indiatimes.com/business/india-business/indias-exports-to-us-could-face-5-76-billion-decline-in-2025-due-to-tariff-hikes/articleshow/120059905.cms",
    "category": "economic impact"
  },
  {
    "id": "007",
    "event": "China Plans High-Level Meeting to Address US Tariffs",
    "date": "2025-04-09",
    "region": "China",
    "summary": "China Plans High-Level Meeting to Address US Tariffs. Chinese officials are convening a high-level meeting to craft economic responses following the US’s steep tariff hike.",
    "source": "https://www.reuters.com/world/china/china-hold-high-level-meeting-response-us-tariffs-say-sources-2025-04-09/",
    "category": "government response"
  },
  {
    "id": "008",
    "event": "India Pursues New Trade Deals Amid Global Tariff Uncertainty",
    "date": "2025-04-08",
    "region": "India",
    "summary": "India Pursues New Trade Deals Amid Global Tariff Uncertainty. India is accelerating trade talks with several nations to hedge against the impact of rising global tariffs.",
    "source": "https://www.theguardian.com/world/2025/apr/08/tariffs-driving-india-to-strike-trade-deals-finance-minister-says-ahead-of-uk-talks",
    "category": "trade negotiations"
  },
  {
    "id": "009",
    "event": "China Vows to 'Fight to the End' Against US Tariff Threats",
    "date": "2025-04-08",
    "region": "China",
    "summary": "China Vows to 'Fight to the End' Against US Tariff Threats. Beijing has condemned the latest US tariff actions as 'blackmail' and pledged to escalate its resistance.",
    "source": "https://www.theguardian.com/world/2025/apr/08/china-vows-to-fight-to-the-end-against-latest-trump-tariff-threat",
    "category": "diplomatic relations"
  },
  {
    "id": "010",
    "event": "US Stock Markets Volatile Amid Tariff Announcements",
    "date": "2025-04-08",
    "region": "USA",
    "summary": "US Stock Markets Volatile Amid Tariff Announcements. Major US stock indexes fluctuated sharply as new tariff deadlines loomed, reflecting investor nervousness.",
    "source": "https://www.investors.com/market-trend/stock-market-today/dow-jones-sp500-nasdaq-nvidia-stock-nvda-tsla-pltr-hum/",
    "category": "financial markets"
  },
  {
    "id": "011",
    "event": "US Warns of Potential Tariffs on Chinese Steel",
    "date": "2025-03-14",
    "region": "USA",
    "summary": "US Warns of Potential Tariffs on Chinese Steel. The Biden administration has signaled intent to impose new tariffs on Chinese steel amid ongoing trade imbalances.",
    "source": "https://www.nytimes.com/2025/03/14/us-politics/us-tariff-warning-chinese-steel.html",
    "category": "trade policy"
  },
  {
    "id": "012",
    "event": "China Reduces Tariff on Select European Tech Products",
    "date": "2025-02-28",
    "region": "China",
    "summary": "China Reduces Tariff on Select European Tech Products. In a move to diversify trade partnerships, China has lowered tariffs on certain European tech goods.",
    "source": "https://www.dw.com/en/china-eu-tech-tariffs-reduction-2025/a-60321547",
    "category": "trade negotiations"
  },
  {
    "id": "013",
    "event": "India Imposes Temporary Export Ban on Agricultural Products",
    "date": "2025-02-20",
    "region": "India",
    "summary": "India Imposes Temporary Export Ban on Agricultural Products. Due to domestic shortages, India has temporarily halted the export of key agricultural commodities.",
    "source": "https://www.livemint.com/news/india-export-ban-agriculture-commodities-2025-02-20.html",
    "category": "domestic policy"
  }
]


In [5]:
import json
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings


from langchain.chains import RetrievalQA
from langchain.agents import Tool, initialize_agent

from langchain.agents.agent_types import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
documents = []
for item in data:
    content = f"""
    EVENT: {item['event']}
    DATE: {item['date']}
    REGION: {item['region']}
    SUMMARY: {item['summary']}
    SOURCE: {item['source']}
    CATEGORY: {item['category']}
    """
    documents.append(Document(page_content=content.strip(), metadata={"source": item["source"]}))

# ✂️ Chunk the documents
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)


In [7]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory="./chroma_geo_db"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [8]:
# # 🔄 Reload Chroma later
# db = Chroma(
#     persist_directory="./chroma_store",
#     embedding_function=embedding_model
# )

# # ❓ Ask a question
# query = "What did the US do with Chinese EV tariffs?"
# results = db.similarity_search(query, k=3)

# # 📄 Print answers
# for i, doc in enumerate(results):
#     print(f"\nResult {i+1}:\n", doc.page_content)

In [39]:
from collections import defaultdict

used_tools = defaultdict(bool)
tool_results = {}

def tool_wrapper(name, func):
    def wrapped(input):
        if used_tools[name]:
            return f"Tool {name} has already been used. Use the result below to answer the question:\n\n{tool_results[name]}"
        used_tools[name] = True
        result = func(input)
        tool_results[name] = result
        return result
    return wrapped


In [42]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    convert_system_message_to_human=True,
    google_api_key=os.environ["GOOGLE_API_KEY_GENAI"],
    system_message=(
        "You are a geopolitical analyst. You may only call each tool once per question. "
        "If you've already received information from a tool, do not call it again. "
        "Always include source URLs from tool observations in your final answer."
    )
)
# 🔁 Create RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
# Global cache for source docs with URLs and scores
last_tool_sources = {}

def geonewsdb_tool_fn(query: str) -> str:
    response = rag_chain(query)
    docs = response.get("source_documents", [])
    last_tool_sources["GeoNewsDB"] = docs

    # Extract URLs if available
    urls = []
    for doc in docs:
        metadata = doc.metadata or {}
        url = metadata.get("source") or metadata.get("url")
        if url:
            urls.append(url)

    # Compose response
    answer = response["result"]
    sources = "\n".join(f"- {url}" for url in urls) if urls else "No sources available."

    return f"""According to GeoNewsDB: {answer}
"""
    
# geonews_tool = Tool(
#     name="GeoNewsDB",
#     func=geonewsdb_tool_fn,
#     description="Use this for geopolitical events from trusted sources, especially historical or documented events for past dates."
# )

# tools = [
#     geonews_tool,
#     Tool(
#     name="GoogleSearch",
#     func=search.run,
#     description="Useful for answering questions about current events or recent information."
# )
# ]
geo_tool = Tool(
    name="GeoNewsDB",
    func=tool_wrapper("GeoNewsDB", geonewsdb_tool_fn),
    description="Use for known, documented events before April 20, 2025."
)

google_tool = Tool(
    name="GoogleSearch",
    func=tool_wrapper("GoogleSearch", search.run),
    description="Use for real-time updates on or after April 20, 2025."
)

tools = [geo_tool, google_tool]



In [51]:
from langchain.agents import Tool
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Step 3: Define the few-shot ReAct prompt
few_shot_prompt = PromptTemplate.from_template("""
You are a smart assistant that answers questions about tariffs and geopolitics. You have access to two tools:

- Use **GeoNewsDB** only once per question, for known or historical events before the present date.
- Use **GoogleSearch** only once per question, and only for real-time developments on/after present date.
# - If a question may require both tools, use each **only once**, and combine their results.

When providing your final answer:"<clear, neutral summary of the facts>"

Guidelines:
- Do not hallucinate — if uncertain, say so in the "answer".
- Extract all relevant info from the first tool call. Do not repeat tool calls.
- If a tool was already called, do not use it again.
- Always use the observation to form your final answer.
- Be concise, accurate, and trustworthy.

---

Examples:

Question: Did India cut repo rates in 2020?  
Thought: This is historical. I will use GeoNewsDB.  
Action: GeoNewsDB  
Action Input: India repo rate 2020  
Observation: India reduced its repo rate by 75 basis points in March 2020.  
Final Answer:"Yes, India reduced its repo rate by 75 basis points in March 2020."

---

Question: Has the US recently increased tariffs on Chinese EVs?  
Thought: This is recent. I will use GoogleSearch.  
Action: GoogleSearch  
Action Input: US tariff Chinese EVs 2025  
Observation: On April 8, 2025, the US imposed a 104% tariff on Chinese electric vehicles.  
Final Answer:"Yes, the US imposed a 104% tariff on Chinese electric vehicles in April 2025."

---

Question: Did the US warn about new Chinese steel tariffs in March 2025?  
Thought: This is a historical event from March. I will use GeoNewsDB.  
Action: GeoNewsDB  
Action Input: US Chinese steel tariff warning March 2025  
Observation: In March 2025, the US warned of potential new tariffs on Chinese steel amid ongoing trade imbalances.  
Final Answer:"Yes, in March 2025, the US warned of potential new tariffs on Chinese steel."

---

Question: Did China reduce tariffs on European tech products in early 2025?  
Thought: This is a past event from February. I will use GeoNewsDB.  
Action: GeoNewsDB  
Action Input: China tariff cut European tech February 2025  
Observation: In February 2025, China reduced tariffs on certain European tech products to diversify trade partnerships.  
Final Answer:"Yes, in February 2025, China reduced tariffs on certain European tech products to diversify trade partnerships."

---

Now answer this new question:  
Question: {input}  
{agent_scratchpad}

""")

# Step 4: Build agent manually
from langchain.agents import initialize_agent, AgentType

# Initialize the agent with the few-shot prompt
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prompt=few_shot_prompt,
    verbose=True,
    max_iterations=3,
)


In [52]:
def reset_tool_usage():
    used_tools.clear()

# Before each agent run
reset_tool_usage()

In [53]:
# 🧪 Query the agent
# query = "Did the US impose a 104% tariff on Chinese electric vehicles recently?"
# query = "Did India cut the repo rate in April 2025?"
# query = "Is india retaliating like china or trying to resolve the issue by discussion with US on tariffs?"
# query ="Why was the US markets volatile on 2025-04-08?"

query ="Did India take any export-related actions in early 2025?"
response = agent.invoke({"input": query})
# 🔍 Print the full response to check keys
print("\n🧪 Raw Response:\n", response)

# ✅ Then extract the correct field based on available keys
if "result" in response:
    print("\n✅ Final Answer:\n", response["result"])
elif "output" in response:
    print("\n✅ Final Answer:\n", response["output"])
else:
    print("\n❓ Couldn't find expected result. Here's the full response again:")
    print(response)



> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:388: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Question: Did India take any export-related actions in early 2025?
Thought: I should check GeoNewsDB for events related to Indian exports in early 2025.
Action: GeoNewsDB
Action Input: India export policy changes January 2025 to March 2025

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:388: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Observation: According to GeoNewsDB: India temporarily halted the export of key agricultural commodities in February 2025 due to domestic shortages.

Thought:

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:388: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: I have found relevant information about India's export actions in early 2025.
Final Answer: Yes, India temporarily halted the export of key agricultural commodities in February 2025 due to domestic shortages.

> Finished chain.

🧪 Raw Response:
 {'input': 'Did India take any export-related actions in early 2025?', 'output': 'Yes, India temporarily halted the export of key agricultural commodities in February 2025 due to domestic shortages.'}

✅ Final Answer:
 Yes, India temporarily halted the export of key agricultural commodities in February 2025 due to domestic shortages.


In [14]:
# if "sources" in last_tool_sources.get("GeoNewsDB", {}):
#     sources = "\n".join(last_tool_sources["GeoNewsDB"]["sources"])
#     final_answer += f"\n\nSources:\n{sources}"

# # print(final_answer)
# print(last_tool_sources)


# Extract URLs if available
urls = []
for doc in last_tool_sources["GeoNewsDB"]:
    metadata = doc.metadata or {}
    url = metadata.get("source") or metadata.get("url")
    if url:
        urls.append(url)
print("FINAL RESPONSE: "+response["output"]+ " \nSources: " + "\n".join(urls))


FINAL RESPONSE: Yes, India temporarily halted the export of key agricultural commodities in February 2025 due to domestic shortages. 
Sources: https://www.livemint.com/news/india-export-ban-agriculture-commodities-2025-02-20.html
https://timesofindia.indiatimes.com/business/india-business/indias-exports-to-us-could-face-5-76-billion-decline-in-2025-due-to-tariff-hikes/articleshow/120059905.cms


In [ ]:
!pip install -U tqdm pandas datasets

In [15]:
!pip install -U torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.5 MB/s eta 0:00:00:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 18.4 MB/s eta 0:00:0000:010:01
   ━━━

In [16]:
%reload_ext autoreload
%autoreload 2

In [17]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

In [18]:
from huggingface_hub import notebook_login

notebook_login()

In [19]:
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(
        page_content=item["summary"],               # or you could use `item["event"] + " " + item["summary"]`
        metadata={"source": item["source"]}         # keeping source for traceability
    )
    for item in data
]

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""]
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

In [21]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.text_generation(
        prompt=prompt,
        max_new_tokens=1000,
        temperature=0.7,
        return_full_text=False
    )
    return response

In [22]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [24]:
import random

N_GENERATIONS = 2  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=sampled_context.page_content))
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "reference_answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

Generating 2 QA couples...


  0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
display(pd.DataFrame(outputs).head(3))

,context,question,reference_answer,source_doc
0,US Warns of Potential Tariffs on Chinese Steel. The Biden administration has signaled intent to impose new tariffs on Chinese steel amid ongoing trade imbalances.,Is the US planning to impose new tariffs on Chinese steel?\n,"Yes, the Biden administration has signaled intent to impose new tariffs on Chinese steel amid ongoing trade imbalances.",https://www.nytimes.com/2025/03/14/us-politics/us-tariff-warning-chinese-steel.html
1,China Plans High-Level Meeting to Address US Tariffs. Chinese officials are convening a high-level meeting to craft economic responses following the US’s steep tariff hike.,What are Chinese officials planning to do after the US tariff hike?\n,Chinese officials are planning to convene a high-level meeting to craft economic responses following the US’s steep tariff hike.,https://www.reuters.com/world/china/china-hold-high-level-meeting-response-us-tariffs-say-sources-2025-04-09/


In [26]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [27]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        continue

Generating critique for each QA couple...


  0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
generated_questions = pd.DataFrame.from_dict(outputs)
display(
    generated_questions[
        [
            "question",
            "reference_answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

,question,reference_answer,groundedness_score,relevance_score,standalone_score
0,Is the US planning to impose new tariffs on Chinese steel?\n,"Yes, the Biden administration has signaled intent to impose new tariffs on Chinese steel amid ongoing trade imbalances.",5,1,4
1,What are Chinese officials planning to do after the US tariff hike?\n,Chinese officials are planning to convene a high-level meeting to craft economic responses following the US’s steep tariff hike.,3,1,5


In [29]:
list1=generated_questions.to_dict(orient="records")

In [30]:
list1

[{'context': 'US Warns of Potential Tariffs on Chinese Steel. The Biden administration has signaled intent to impose new tariffs on Chinese steel amid ongoing trade imbalances.',
  'question': 'Is the US planning to impose new tariffs on Chinese steel?\n',
  'reference_answer': 'Yes, the Biden administration has signaled intent to impose new tariffs on Chinese steel amid ongoing trade imbalances.',
  'source_doc': 'https://www.nytimes.com/2025/03/14/us-politics/us-tariff-warning-chinese-steel.html',
  'groundedness_score': 5,
  'groundedness_eval': 'The context clearly states that the US is considering imposing new tariffs on Chinese steel, and the administration has signaled its intent to do so. Therefore, the answer to the question is straightforwardly yes.\n\n',
  'relevance_score': 1,
  'relevance_eval': 'The question is about international trade policy, not machine learning or NLP. It does not involve any aspect of the Hugging Face ecosystem.\n\n',
  'standalone_score': 4,
  'stan

In [ ]:
# list1=[{'context': 'US Warns of Potential Tariffs on Chinese Steel. The Biden administration has signaled intent to impose new tariffs on Chinese steel amid ongoing trade imbalances.',
#   'question': 'Is the US planning to impose new tariffs on Chinese steel?\n',
#   'reference_answer': 'Yes, the Biden administration has signaled intent to impose new tariffs on Chinese steel amid ongoing trade imbalances.',
#   'source_doc': 'https://www.nytimes.com/2025/03/14/us-politics/us-tariff-warning-chinese-steel.html',
#   'groundedness_score': 5,
#   'groundedness_eval': 'The context clearly states that the US is considering imposing new tariffs on Chinese steel, and the administration has signaled its intent to do so. Therefore, the answer to the question is straightforwardly yes.\n\n',
#   'relevance_score': 1,
#   'relevance_eval': 'The question is about international trade policy, not machine learning or NLP. It does not involve any aspect of the Hugging Face ecosystem.\n\n',
#   'standalone_score': 4,
#   'standalone_eval': 'This question is about international trade policy and is specific and self-contained, though it does refer to a current or recent event that may affect its relevance.\n'},
#  {'context': 'China Plans High-Level Meeting to Address US Tariffs. Chinese officials are convening a high-level meeting to craft economic responses following the US’s steep tariff hike.',
#   'question': 'What are Chinese officials planning to do after the US tariff hike?\n',
#   'reference_answer': 'Chinese officials are planning to convene a high-level meeting to craft economic responses following the US’s steep tariff hike.',
#   'source_doc': 'https://www.reuters.com/world/china/china-hold-high-level-meeting-response-us-tariffs-say-sources-2025-04-09/',
#   'groundedness_score': 3,
#   'groundedness_eval': 'The context explicitly states that Chinese officials are planning to address the US tariffs by holding a high-level meeting, so it is clear that they are planning to respond to the tariff hike in some way. However, the context does not provide any details about what their specific plans are for addressing the tariffs. Therefore, while the context confirms that they are planning to do something, it does not provide enough information to unambiguously answer the question about what they are planning to do specifically.\n',
#   'relevance_score': 1,
#   'relevance_eval': 'This question is not directly related to machine learning, natural language processing, or the Hugging Face ecosystem. Therefore, it has low utility for machine learning developers building NLP applications with Hugging Face.\n\n',
#   'standalone_score': 5,
#   'standalone_eval': 'The question is clear about what is being asked - the response is expected to mention what Chinese officials are planning to do as a reaction to a specific event (US tariff hike).\n'}]

In [31]:
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 0.5)
    & (generated_questions["relevance_score"] >= 0.5)
    & (generated_questions["standalone_score"] >= 0.5)
]
display(
    generated_questions[
        [
            "question",
            "reference_answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

,question,reference_answer,groundedness_score,relevance_score,standalone_score
0,Is the US planning to impose new tariffs on Chinese steel?\n,"Yes, the Biden administration has signaled intent to impose new tariffs on Chinese steel amid ongoing trade imbalances.",5,1,4
1,What are Chinese officials planning to do after the US tariff hike?\n,Chinese officials are planning to convene a high-level meeting to craft economic responses following the US’s steep tariff hike.,3,1,5


In [32]:
single_question_df = generated_questions.iloc[[0]]  # double brackets to keep it a DataFrame

# Convert to Hugging Face dataset
eval_dataset = datasets.Dataset.from_pandas(single_question_df, split="train", preserve_index=False)

In [54]:
reset_tool_usage()

In [55]:
from tqdm import tqdm

# Add a column for model-generated answers
generated_answers = []
for example in tqdm(eval_dataset):
    question = example["question"]
    generated = agent.invoke({"input": question}) # or agent.invoke()
    
    # Handle string/JSON depending on your chain output format
    if isinstance(generated, dict) and "output" in generated:
        answer = generated["output"]
    else:
        answer = str(generated)
    
    generated_answers.append(answer)

# Add answers to dataset
print(generated_answers)
single_question_df["generated_answer"] = generated_answers
eval_dataset = datasets.Dataset.from_pandas(single_question_df, split="train", preserve_index=False)


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:388: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")




> Entering new AgentExecutor chain...
Thought: I need to find out if the US is planning to impose new tariffs on Chinese steel. This could be a recent event, so I should check recent news.  Since the current date is past April 20, 2025, I will use GoogleSearch.

Action: GoogleSearch
Action Input: "US planning new tariffs Chinese steel"
Observation: Jun 20, 2024 ... ... steel, aluminum, and Chinese tariffs to US prices. It also found an ... U.S. confirms announced July 5 plans to impose tariffs on all Chinese imports ... Hong Kong's leader John Lee condemned the US move and announced plans to file a complaint with the World Trade Organization, a step China has already taken. As ... 2 days ago ... Days after taking office, Trump imposes a new 10% tariff on all Chinese imports. That's on top of existing tariffs from his first term and from the Biden era. Apr 19, 2024 ... ... United States Trade Representative ("USTR") will (1) raise the current Section 301 tariffs on Chinese steel and a

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:388: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Based on the Google Search results, there have been past tariffs on Chinese steel and aluminum, and discussions of new tariffs. However, the results don't definitively say new tariffs are currently *planned*.  I need to refine my search.

Action: GoogleSearch
Action Input: "US new tariffs Chinese steel 2025 or later"
Observation: Tool GoogleSearch has already been used. Use the result below to answer the question:

Jun 20, 2024 ... ... steel, aluminum, and Chinese tariffs to US prices. It also found an ... U.S. confirms announced July 5 plans to impose tariffs on all Chinese imports ... Hong Kong's leader John Lee condemned the US move and announced plans to file a complaint with the World Trade Organization, a step China has already taken. As ... 2 days ago ... Days after taking office, Trump imposes a new 10% tariff on all Chinese imports. That's on top of existing tariffs from his first term and from the Biden era. Apr 19, 2024 ... ... United States Trade Representative ("U

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:388: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
100%|██████████| 1/1 [00:08<00:00,  8.05s/it]

Thought: The refined search didn't provide any information about new tariffs on Chinese steel planned for 2025 or later.  The most recent information is from early 2025, and refers to tariffs already in place.  I can't find confirmation of *planned* tariffs.

Final Answer: Based on available information, there is no confirmation that the US is *planning* to impose new tariffs specifically on Chinese steel in 2025 or later.  There have been tariffs imposed in the past, and some discussions of potential future tariffs, but no concrete plans have been announced.

> Finished chain.
['Based on available information, there is no confirmation that the US is *planning* to impose new tariffs specifically on Chinese steel in 2025 or later.  There have been tariffs imposed in the past, and some discussions of potential future tariffs, but no concrete plans have been announced.']



/tmp/ipykernel_31/503803612.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_question_df["generated_answer"] = generated_answers


In [58]:
display(pd.DataFrame(eval_dataset).head(3))

,context,question,reference_answer,source_doc,groundedness_score,groundedness_eval,relevance_score,relevance_eval,standalone_score,standalone_eval,generated_answer
0,US Warns of Potential Tariffs on Chinese Steel. The Biden administration has signaled intent to impose new tariffs on Chinese steel amid ongoing trade imbalances.,Is the US planning to impose new tariffs on Chinese steel?\n,"Yes, the Biden administration has signaled intent to impose new tariffs on Chinese steel amid ongoing trade imbalances.",https://www.nytimes.com/2025/03/14/us-politics/us-tariff-warning-chinese-steel.html,5,"The context clearly states that the US is considering imposing new tariffs on Chinese steel, and the administration has signaled its intent to do so. Therefore, the answer to the question is straightforwardly yes.\n\n",1,"The question is about international trade policy, not machine learning or NLP. It does not involve any aspect of the Hugging Face ecosystem.\n\n",4,"This question is about international trade policy and is specific and self-contained, though it does refer to a current or recent event that may affect its relevance.\n","Based on available information, there is no confirmation that the US is *planning* to impose new tariffs specifically on Chinese steel in 2025 or later. There have been tariffs imposed in the past, and some discussions of potential future tariffs, but no concrete plans have been announced."


In [59]:
criteria = {
    "correctness": "Is the answer factually accurate?",
    "relevance": "Is the answer relevant to the question?",
    "conciseness": "Is the answer concise and to the point?"
}

In [65]:
from langchain.evaluation import load_evaluator
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    google_api_key=os.environ["GOOGLE_API_KEY_GENAI"]
)

criteria = {
    "correctness": "Is the answer factually accurate based on the reference?",
    "relevance": "Does the answer address the question appropriately?",
    "conciseness": "Is the answer free of unnecessary information?"
}

evaluator = load_evaluator(
    "criteria",
    criteria=criteria,
    llm=llm,
)

results = []
for row in eval_dataset:
    eval_result = evaluator.evaluate_strings(
        input=row["question"],
        prediction=row["generated_answer"],
        reference=row["reference_answer"],
    )
    results.append(eval_result)

# Store in your dataframe
single_question_df["evaluation"] = results


/usr/local/lib/python3.11/dist-packages/langchain/evaluation/schema.py:129: UserWarning: Ignoring reference in CriteriaEvalChain, as it is not expected.
To use references, use the labeled_criteria instead.
  warn(self._skip_reference_warning)
/tmp/ipykernel_31/1954221723.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_question_df["evaluation"] = results


In [69]:
display(single_question_df['evaluation'])

0    {'reasoning': '* **Correctness:** The submission acknowledges past tariffs and the possibility of future ones, but correctly states there's no confirmation of *planned* new tariffs specifically on Chinese steel.  This hedges appropriately and avoids definitive claims without sufficient evidence.  Assuming the reference information doesn't contain any announcements of planned tariffs, the submission is factually accurate.

* **Relevance:** The submission directly addresses the question about planned US tariffs on Chinese steel. It doesn't stray into unrelated topics.

* **Conciseness:** The answer is concise. It provides the necessary information without unnecessary detail or background.  The phrasing is efficient and to the point.


Y', 'value': 'Y', 'score': 1}
Name: evaluation, dtype: object